In [ ]:
%config InlineBackend.figure_formats = ['svg']
import quimb as qu
import quimb.tensor as qtn
import numpy as np

In [ ]:
n = 6
gate2 = 'CZ'

# the hamiltonian
H = qu.ham_ising(n, jz=1.0, bx=0.7, cyclic=False)

# the propagator for the hamiltonian
t = 2
U_dense = qu.expm(-1j * t * H)

# 'tensorized' version of the unitary propagator
U = qtn.Tensor(
    data=U_dense.reshape([2] * (2 * n)),
    inds=[f'k{i}' for i in range(n)] + [f'b{i}' for i in range(n)],
    tags={'U_TARGET'}
)
U.draw(color=[ 'U_TARGET','MPO'])

chi = [2,2,3,3,2,2]

d = 2
tn_guess = qtn.TensorNetwork([
    qtn.Tensor(np.random.normal(size=(d, d, chi[0], chi[1])), inds=(f'b{0}',f'k{0}',f'l{0}',f'l{1}' ),tags={'MPO'}),
    qtn.Tensor(np.random.normal(size=(d, d, chi[1], chi[2])), inds=(f'b{1}',f'k{1}',f'l{1}',f'l{2}' ),tags={'MPO'}),
    qtn.Tensor(np.random.normal(size=(d, d, chi[2], chi[3])), inds=(f'b{2}',f'k{2}',f'l{2}',f'l{3}' ),tags={'MPO'}),
    qtn.Tensor(np.random.normal(size=(d, d, chi[3], chi[4])), inds=(f'b{3}',f'k{3}',f'l{3}',f'l{4}' ),tags={'MPO'}),
    qtn.Tensor(np.random.normal(size=(d, d, chi[4], chi[5])), inds=(f'b{4}',f'k{4}',f'l{4}',f'l{5}' ),tags={'MPO'}),
    qtn.Tensor(np.random.normal(size=(d, d, chi[5], chi[0])), inds=(f'b{5}',f'k{5}',f'l{5}',f'l{0}' ),tags={'MPO'})
])
tn_guess.draw(color=[ 'U_TARGET','MPO'])

(tn_guess.H & U).draw(color=['U_TARGET','MPO'])

In [ ]:
def normalize_op(mpo):
    mpo /= mpo.norm()
    mpo *= np.sqrt(2**n)
    return mpo

In [ ]:
def negative_overlap(mpo, U):
    return - abs((mpo.H & U).contract(all, optimize='auto-hq')) / 2**n #(mpo.H @ u_target)  # minus so as to minimize  


In [ ]:
def loss(V, U):
    return 1 - abs((V.H & U).contract(all, optimize='auto-hq')) / 2**n

In [ ]:
from quimb.tensor.optimize import TNOptimizer

optmzr = TNOptimizer(
    tn_guess,                           # our initial input, the tensors of which to optimize
    loss_fn=negative_overlap,
    norm_fn=normalize_op,
    loss_constants={'U': U},            # this is a constant TN to supply to loss_fn
    autodiff_backend='tensorflow',      # {'jax', 'tensorflow', 'autograd'}
    optimizer='L-BFGS-B',               # supplied to scipy.minimize
)
mpo_opt = optmzr.optimize(100)

In [ ]:
tnopt = qtn.TNOptimizer(
    tn_guess,               # the tensor network we want to optimize
    loss,                     # the function we want to minimize
    loss_constants={'U': U},  # supply U to the loss function as a constant TN
    tags=['MPO'],             # only optimize U3 tensors
    autodiff_backend='jax',   # use 'autograd' for non-compiled optimization
    optimizer='L-BFGS-B',     # the optimization algorithm
)

In [ ]:
# allow 10 hops with 500 steps in each 'basin'
tn_opt = tnopt.optimize_basinhopping(n=500, nhop=10)

In [ ]:
tn_opt_dense = tn_opt.to_dense([f'k{i}' for i in range(n)], [f'b{i}' for i in range(n)])

psi0 = qu.rand_ket(2**n)

# this is the exact state we want
psif_exact = U_dense @ psi0

# this is the state our circuit will produce if fed `psi0`
psif_apprx = tn_opt_dense @ psi0

f"Fidelity: {100 * qu.fidelity(psif_apprx, psif_exact):.2f} %"